In [28]:
from bs4 import BeautifulSoup
from urllib.request import urlopen as uReq
url = "http://forem.co.in/members.html"
# Make a GET request to fetch the raw HTML content
html_content = uReq(url)

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")
#print(soup.prettify()) # print the parsed data of html

from collections import defaultdict
data = {}
table1 = soup.find("table", attrs={"class": "table"})
table_data = table1.findAll("tr")  # contains 71 rows
table_data[1]
# Get all the headings of Lists
headings = []
for th in table_data[0].findAll("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(th.text.replace('\n', ' '))

t_headers = []

for i in range(len(table_data)):
    for th in table_data[i].findAll("td"):
        # remove any newlines and extra spaces from left and right
        t_headers.append(th.text.replace('\n', ' ').strip())
        
res=[]
for i in range(0,len(t_headers),len(headings)):
    res.append(t_headers[i:i+5])
temp = defaultdict(list)
for i in range(len(res)):
    class_list = headings
    id_list = res[i]
    for key, value in zip(class_list, id_list):
        temp[key].append(value)
headings

['S.No.', 'Name of Member', 'Event Company', 'Contact No.', 'Email ID']

In [29]:
# print the contents using zip format. 
res=[]
for each_row in zip(*([i] + (j) for i, j in temp.items())): 
      res.append((*each_row,))
    
# Using pandas to clean data    
import pandas as pd
df=pd.DataFrame(res)
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

data=df
data.head()

,S.No.,Name of Member,Event Company,Contact No.,Email ID
1,1,Aayush Gupta,Anehi Wedding Plaaner Pvt Ltd,7597120888,anehiweddingplanner@gmail.com
2,2,Aditya Viay,Occasion Events,9680361401,adityavj31@gmail.com
3,3,Ajay Agarwal,Arch Promotions & Services,9829012210,ajayagarwal1966@gmail.com
4,4,Ajay Chouhan,Challengers Event,9351434630,challengersevents@gmail.com
5,5,Amit Dogra,Royal India Wedding,9929106781,amit@royalindiawedding.com


In [30]:
# dropping null value columns to avoid errors 
data.dropna(inplace = True) 
#Renaming old column names
data["Organization Name"]=data["Event Company"]
data["Contact Person Name"]=data["Name of Member"]
# Dropping old columns 
data.drop(columns =["Event Company"], inplace = True) 
data.drop(columns =["Name of Member"], inplace = True) 
# new data frame with split value columns 
newphone = data["Contact No."].str.split("/", n = 1, expand = True)  
# making separate column from new data frame 
data["Phone - Mobile"]= newphone[0] 
data["Phone 2- Mobile"]= newphone[1] 
# Dropping old Contact No. column 
data.drop(columns =["Contact No."], inplace = True) 

# new data frame with split value columns 
newEmail = data["Email ID"].str.split(",", n = 1, expand = True)  
# making separate column from new data frame 
data["E-mail 1"]= newEmail[0] 
data["E-mail 2"]= newEmail[1] 
# Dropping old Email ID columns 
data.drop(columns =["Email ID"], inplace = True) 
data[19:25]

,S.No.,Organization Name,Contact Person Name,Phone - Mobile,Phone 2- Mobile,E-mail 1,E-mail 2
20,20,Mr. Verma Events and Entertainment,Gaurav Verma,8384978567,None,mrvermaeventsandentertainment@gmail.com,None
21,21,Rising Events & Promotions Pvt Ltd.,Harpreet Singh,9887388800,None,harpreet@risingevents.org,None
22,22,Samrock India Event & Entertainment,Harsh Samaria,9649920390,None,info@samrockindia.com,samrockindiaevent@gmail.com
23,23,Aaradhya events & marketing,Hitendra Sharma,9829075422,None,hitendra_events@gmail.com,None
24,24,Moments of Life,Kartik Tahiliani,8955570109,None,momentsoflifeweddingplanners@gmail.com,None
25,25,Red M Exhibitions and Events (P) Ltd.,Kuldeep Singh,9929544466,None,redmevents@gmail.com,None


In [31]:
# Data frame to excel  
df.to_excel('Members.xlsx', index = False, header=True) 